In [1]:
import sys
import os
from shapely import speedups
speedups.disable()
from shapely.geometry import shape
from shapely.ops import nearest_points
import fiona
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from mpl_toolkits.axes_grid1 import make_axes_locatable
from zipfile import ZipFile
from shapely import wkt
import branca
import json
from geopandas import GeoDataFrame
from shapely.geometry import Point
import glob
import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_colwidth', 800)

In [2]:
import math
class Haversine:
    '''
    use the haversine class to calculate the distance between
    two lon/lat coordnate pairs.
    output distance available in kilometers, meters, miles, and feet.
    example usage: Haversine([lon1,lat1],[lon2,lat2]).feet
    
    '''
    def __init__(self,coord1,coord2):
        lon1,lat1=coord1
        lon2,lat2=coord2
        
        R=6371000                               # radius of Earth in meters
        phi_1=math.radians(lat1)
        phi_2=math.radians(lat2)

        delta_phi=math.radians(lat2-lat1)
        delta_lambda=math.radians(lon2-lon1)

        a=math.sin(delta_phi/2.0)**2+\
           math.cos(phi_1)*math.cos(phi_2)*\
           math.sin(delta_lambda/2.0)**2
        c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
        
        self.meters=R*c                         # output distance in meters
        self.km=self.meters/1000.0              # output distance in kilometers
        self.miles=self.meters*0.000621371      # output distance in miles
        self.feet=self.miles*5280               # output distance in feet

if __name__ == "__Haversine__":
    main()

In [10]:
def get_all_stations_info():
    # you can see it in yoru briwser by going to: 
#   # https://gbfs.urbansharing.com/edinburghcyclehire.com/station_information.json
    response_station_information = requests.request("GET", "http://chargepoints.dft.gov.uk/api/retrieve/registry/format/json/")
    stations_info = response_station_information.json()
    return stations_info

In [24]:
class Station:
    def __init__(self, dictionary_from_api):
        self.id= dictionary_from_api['ChargeDeviceId']
        self.post_code = dictionary_from_api['ChargeDeviceLocation']['Address']['PostCode']
        self.latitude = dictionary_from_api['ChargeDeviceLocation']['Latitude']  if dictionary_from_api['ChargeDeviceLocation']['Latitude'] != None else 0
        self.longitude = dictionary_from_api['ChargeDeviceLocation']['Longitude']  if dictionary_from_api['ChargeDeviceLocation']['Longitude']!= None else 0
        self.device_network = dictionary_from_api['DeviceNetworks']
        self.org_name =dictionary_from_api['DeviceOwner']['OrganisationName']
        self.on_street = dictionary_from_api['OnStreetFlag']
        self.location_type = dictionary_from_api['LocationType']
        self.connectors = []
        self.connector_type = []
        self.connector_kw = []
        self.Number_of_Connector = 0
        self.connector_merged_kw =[]
        # if dictionary_from_api['date_of_creation'] != None else 0
    def add_connector(self, connector):
        self.connectors.append(connector)
    def add_connectortype(self, connectortype):
        self.connector_type.append(connectortype)
    def add_connectorKW(self, connectorkw):
        self.connector_kw.append(connectorkw)
    def assign_numberofconnector(self, connetor_number):
        self.Number_of_Connector = connetor_number    
    def add_merged_connector_kw(self, list1 , list2):
        self.connector_merged_kw.append(" ".join([list1,list2]))
        
def data_into_objects(list_with_company_dictionaries):
    station_objects = [Station(station_dictionary) 
                       for station_dictionary in list_with_company_dictionaries]
    return station_objects
def prepare_data(station_objects):      
    return { 'station_id': [ station.id for station in station_objects ],
             'device_network': [ station.device_network for station in station_objects ],
             'organisation_name': [station.org_name for station in station_objects],
             'on_street':[station.on_street for station in station_objects],
             'location_type': [station.location_type for station in station_objects],
             'post_code' :[ station.post_code for station in station_objects ],
            'latitude' :[ station.latitude for station in station_objects ],
             'longitude': [ station.longitude for station in station_objects ],
              'Connector': [ ", ".join(str(x) for x in station.connectors) for station in station_objects ],
            'Connector_Type': [ ", ".join(str(x) for x in station.connector_type) for station in station_objects ],
            'Connector_KW': [ ", ".join(str(x) for x in station.connector_kw) for station in station_objects ],
            'Number_of_Connector' : [ station.Number_of_Connector for station in station_objects ],
            'Connector_Type_List': [ station.connector_type for station in station_objects ],
            'Connector_Type_Merged_KW': [ station.connector_merged_kw for station in station_objects ],
           }
def charger_info(station_objects,list_with_company_dictionaries):
    for i , object in enumerate(station_objects):
        connetor_number = len(list_with_company_dictionaries[i]['Connector'])
        object.assign_numberofconnector(connetor_number)
        if connetor_number > 0:
            for y in range(0,connetor_number):
                object.add_connector(list_with_company_dictionaries[i]['Connector'][y].get('ChargeMethod'))
                object.add_connectortype(list_with_company_dictionaries[i]['Connector'][y].get('ConnectorType'))
                object.add_connectorKW(list_with_company_dictionaries[i]['Connector'][y].get('RatedOutputkW'))
                object.add_merged_connector_kw(list_with_company_dictionaries[i]['Connector'][y].get('ConnectorType'),list_with_company_dictionaries[i]['Connector'][y].get('RatedOutputkW'))
#                object.add_status(list_with_company_dictionaries[i]['Connector'][y].get('ChargePointStatus'))

In [15]:
slow = ['3-pin Type G (BS1363) 3.7','3-pin Type G (BS1363) 3.0', 'Type 1 SAEJ1772 (IEC 62196) 3.0', 
        'Type 2 Mennekes (IEC62196) 3.7', 'Type 1 SAEJ1772 (IEC 62196) 3.7'] # 5 types 
fast = ['Type 2 Mennekes (IEC62196) 7.0','Type 2 Mennekes (IEC62196) 22.0', 'Type 3 Scame (IEC62196) 22.0',
       'Type 1 SAEJ1772 (IEC 62196) 7.0','Type 2 Mennekes (IEC62196) 11.0', 'Type 3 Scame (IEC62196) 7.0', 
        'Type 2 Combo (IEC62196) DC 22.0'] # 7 types
rapid = ['JEVS G105 (CHAdeMO) DC 50.0','Type 2 Mennekes (IEC62196) 43.0','Type 2 Combo (IEC62196) DC 50.0',
         'JEVS G105 (CHAdeMO) DC 22.0', 'Type 2 Tesla (IEC62196) DC 120.0','JEVS G105 (CHAdeMO) DC 36.0',
       'Type 2 Combo (IEC62196) DC 36.0', 'JEVS G105 (CHAdeMO) DC 100.0',
       'Type 2 Combo (IEC62196) DC 175.0',
       'Type 2 Combo (IEC62196) DC 350.0' ]#10 types

# Creating Data

In [25]:
infos=get_all_stations_info()
station_objects = data_into_objects(infos['ChargeDevice'])
charger_info(station_objects , infos['ChargeDevice'])
Station_data = prepare_data(station_objects)
Station_df = pd.DataFrame(Station_data)

Station_df['Charger_Category'] = Station_df.Connector_Type_Merged_KW.apply(lambda x: "Rapid" if any(item in rapid for item in x) 
                                                                           else ("Fast" if any(item in fast for item in x) 
                                                                           else "Slow"))

In [ ]:
#API return was saved as Json to not call and wait API for everytime run the code 
with open('stations.json') as f:
        data = json.load(f)

In [29]:
Station_df['latitude'] = pd.to_numeric(Station_df['latitude'])
Station_df['longitude'] = pd.to_numeric(Station_df['longitude'])
geometry = [Point(xy) for xy in zip(Station_df.longitude, Station_df.latitude)]
Charging_stations_geo = GeoDataFrame(Station_df, crs="EPSG:4326", geometry=geometry)

In [35]:
uk_border = gpd.read_file("C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Shp_Files\\GBR_adm1.shp")
scotland_border= uk_border[uk_border.NAME_1 == 'Scotland'].reset_index(drop=True)

In [46]:
num = 0 
index_list = []
for i,v in Charging_stations_geo.iterrows(): 
    if scotland_border.geometry[0].contains(v.geometry) == True:
        index_list.append(i)
        num = num + 1

In [52]:
scotland_stations = Charging_stations_geo.iloc[index_list].reset_index(drop=True)

##  Reading Ammenities Data

In [15]:
path_osm='C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\'
all_files_osm = glob.glob(path_osm + "/*.shp")
all_files_osm
# list_of_dfs_osm = [gpd.read_file(filename).to_crs(epsg=4326) for filename in all_files_osm] # I added to_crs part
# list_of_filenames_osm = [find_between(filename, "Open_Data_Geofabric\\",".shp") for filename in all_files_osm]
# dfnames_and_dfvalues_osm = dict(zip(list_of_filenames_osm, list_of_dfs_osm))

['C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_buildings_a_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_landuse_a_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_natural_a_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_natural_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_places_a_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_places_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pofw_a_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pofw_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pois_a_free_1.shp',
 'C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pois_free_1.shp',


In [16]:
natural_osm_poly = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_natural_a_free_1.shp').to_crs("EPSG:4326")
natural_osm_point = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_natural_free_1.shp').to_crs("EPSG:4326")

In [6]:
building_osm = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_buildings_a_free_1.shp').to_crs("EPSG:4326")

In [18]:
natural_osm_poly_elem = natural_osm_poly[natural_osm_poly.name.notnull()].reset_index(drop = True)
natural_osm_point_elem = natural_osm_point[natural_osm_point.name.notnull()].reset_index(drop = True)
building_osm_poly=building_osm[building_osm.name.notnull()].reset_index(drop = True)

In [19]:
landuse_osm = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_landuse_a_free_1.shp').to_crs("EPSG:4326")

In [20]:
pois_osm_poly = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pois_a_free_1.shp').to_crs("EPSG:4326")
pois_osm_point = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pois_free_1.shp').to_crs("EPSG:4326")

In [21]:
pois_osm_poly_elem = pois_osm_poly[pois_osm_poly.name.notnull()].reset_index(drop = True)
pois_osm_point_elem = pois_osm_point[pois_osm_point.name.notnull()].reset_index(drop = True)

In [22]:
Worship_osm_poly = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pofw_a_free_1.shp').to_crs("EPSG:4326")
Worship_osm_point = gpd.read_file('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Open_Data_Geofabric\\gis_osm_pofw_free_1.shp').to_crs("EPSG:4326")

In [23]:
Worship_osm_point_elem = Worship_osm_point[Worship_osm_point.name.notnull()].reset_index(drop = True)
Worship_osm_poly_elem = Worship_osm_poly[Worship_osm_poly.name.notnull()].reset_index(drop=True)

In [24]:
landuse_osm_poly = landuse_osm[landuse_osm.name.notnull()].reset_index(drop=True)

In [25]:
def is_stations_close_poly(dataframe_name, df, id, max_distance):
    scotland_stations[dataframe_name] = ""
    scotland_stations[dataframe_name] = scotland_stations[dataframe_name].astype(object)
    temper = []
    for index, v in scotland_stations.iterrows():
        point= v.geometry
        for j , k in df.iterrows(): 
            near_p = nearest_points(k.geometry,point)
            if Haversine(near_p[0].coords[0], point.coords[0]).km <= max_distance:
                temper.append(df.loc[j,id])
        if len(temper) != 0: 
            scotland_stations_geo[dataframe_name][index] = temper
        temper = []
    return temper

In [26]:
def is_stations_close_point(dataframe_name, df, id, max_distance):
    scotland_stations[dataframe_name] = ""
    scotland_stations[dataframe_name] = scotland_stations[dataframe_name].astype(object)
    temper = []
    for index, v in scotland_stations.iterrows():
        point= v.geometry
        for j , k in df.iterrows(): 
            if Haversine(k.geometry.coords[0], point.coords[0]).km <= max_distance:
                temper.append(df.loc[j,id])
        if len(temper) != 0: 
            scotland_stations_geo[dataframe_name][index] = temper  
        temper = []
    return temper

In [35]:
is_stations_close_poly('Worship_osm_poly_elem',Worship_osm_poly_elem,'osm_id', 3)
is_stations_close_poly('landuse_osm_poly',landuse_osm_poly,'osm_id', 3)
is_stations_close_poly('pois_osm_poly_elem',pois_osm_poly_elem,'osm_id', 3)
is_stations_close_poly('natural_osm_poly_elem',natural_osm_poly_elem,'osm_id', 3)

[]

In [36]:
#scotland_stations_geo.to_csv('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Scotland_Stations_Combined_Last_3.csv', index=None, header=True)

In [37]:
is_stations_close_point('Worship_osm_point_elem',Worship_osm_point_elem,'osm_id', 1)
is_stations_close_point('pois_osm_point_elem',pois_osm_point_elem,'osm_id', 1)
is_stations_close_point('natural_osm_point_elem',natural_osm_point_elem,'osm_id', 3)

[]

In [38]:
#scotland_stations_geo.to_csv('C:\\Users\\yusuf\\Downloads\\Dissertation Code\\Scotland_Stations_Combined_Last_4_Number.csv', index=None, header=True)